In [2]:
#Libraries import
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
#FOURSQUARE RESERVED SECTION
CLIENT_ID = 'WR3G5TKZIDSLK4RHVEIAG03YN1GNHORXEEZ1R01GGUUBQ4E0'
CLIENT_SECRET = 'NTXXAWK2K3X0YNNI0TBJA3QWM5QQPAIEL5K2ZHRUMLW3FVUS'
VERSION = '20190605'

In [3]:
with open('nyu-2451-34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']

In [4]:
#Dataframe initiation
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [5]:
Brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
#display the different neighbourhood in Brooklyn Borough
Brooklyn_data['Neighborhood'].unique().tolist()

['Bay Ridge',
 'Bensonhurst',
 'Sunset Park',
 'Greenpoint',
 'Gravesend',
 'Brighton Beach',
 'Sheepshead Bay',
 'Manhattan Terrace',
 'Flatbush',
 'Crown Heights',
 'East Flatbush',
 'Kensington',
 'Windsor Terrace',
 'Prospect Heights',
 'Brownsville',
 'Williamsburg',
 'Bushwick',
 'Bedford Stuyvesant',
 'Brooklyn Heights',
 'Cobble Hill',
 'Carroll Gardens',
 'Red Hook',
 'Gowanus',
 'Fort Greene',
 'Park Slope',
 'Cypress Hills',
 'East New York',
 'Starrett City',
 'Canarsie',
 'Flatlands',
 'Mill Island',
 'Manhattan Beach',
 'Coney Island',
 'Bath Beach',
 'Borough Park',
 'Dyker Heights',
 'Gerritsen Beach',
 'Marine Park',
 'Clinton Hill',
 'Sea Gate',
 'Downtown',
 'Boerum Hill',
 'Prospect Lefferts Gardens',
 'Ocean Hill',
 'City Line',
 'Bergen Beach',
 'Midwood',
 'Prospect Park South',
 'Georgetown',
 'East Williamsburg',
 'North Side',
 'South Side',
 'Ocean Parkway',
 'Fort Hamilton',
 'Ditmas Park',
 'Wingate',
 'Rugby',
 'Remsen Village',
 'New Lots',
 'Paerdegat Ba

In [6]:
#Foursquare API default function
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    LIMIT=100
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
#Finding the venues for each neighbourhood in Brooklyn
Brooklyn_venues = getNearbyVenues(names=Brooklyn_data['Neighborhood'],
                                   latitudes=Brooklyn_data['Latitude'],
                                   longitudes=Brooklyn_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [8]:
Brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,41,41,41,41,41,41
Bay Ridge,85,85,85,85,85,85
Bedford Stuyvesant,27,27,27,27,27,27
Bensonhurst,34,34,34,34,34,34
Bergen Beach,7,7,7,7,7,7
Boerum Hill,86,86,86,86,86,86
Borough Park,21,21,21,21,21,21
Brighton Beach,43,43,43,43,43,43
Broadway Junction,14,14,14,14,14,14


In [9]:
# one hot encoding reserved
Brooklyn_onehot = pd.get_dummies(Brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")
Brooklyn_onehot['Neighborhood'] = Brooklyn_venues['Neighborhood'] 
fixed_columns = [Brooklyn_onehot.columns[-1]] + list(Brooklyn_onehot.columns[:-1])
Brooklyn_onehot = Brooklyn_onehot[fixed_columns]
Brooklyn_grouped = Brooklyn_onehot.groupby('Neighborhood').mean().reset_index()

In [ ]:
for hood in Brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Brooklyn_grouped[Brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True)) 
    #print('\n') #The last two lines generate lot of data, i have cleared the output for a faster reading.

In [11]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [12]:
num_top_venues = 30

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Brooklyn_grouped['Neighborhood']

for ind in np.arange(Brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
0,Bath Beach,Pharmacy,Sushi Restaurant,Italian Restaurant,Kids Store,Pizza Place,Bubble Tea Shop,Fast Food Restaurant,Chinese Restaurant,Surf Spot,Gas Station,Café,Mobile Phone Shop,Cantonese Restaurant,Women's Store,Clothing Store,Bike Rental / Bike Share,Coffee Shop,Rental Car Location,Donut Shop,Hookah Bar,Pakistani Restaurant,Dessert Shop,Park,Diner,Restaurant,Burger Joint,Peruvian Restaurant,Ice Cream Shop,Bakery,Turkish Restaurant
1,Bay Ridge,Spa,Italian Restaurant,Pizza Place,Bagel Shop,American Restaurant,Greek Restaurant,Bar,Chinese Restaurant,Sandwich Place,Middle Eastern Restaurant,Seafood Restaurant,Playground,Pharmacy,Sushi Restaurant,Ice Cream Shop,Thai Restaurant,Optical Shop,Grocery Store,Hookah Bar,Gym / Fitness Center,Snack Place,Breakfast Spot,Kids Store,Coffee Shop,Pool Hall,Clothing Store,Mobile Phone Shop,Vietnamese Restaurant,Hotel,Juice Bar
2,Bedford Stuyvesant,Coffee Shop,Pizza Place,Café,Deli / Bodega,Bar,Bus Stop,Bagel Shop,New American Restaurant,Gourmet Shop,Cocktail Bar,Boutique,Sandwich Place,Juice Bar,Thrift / Vintage Store,BBQ Joint,Japanese Restaurant,Park,Fried Chicken Joint,Vietnamese Restaurant,Wine Bar,Wine Shop,Farmers Market,Flea Market,Fish Market,Fish & Chips Shop,Flower Shop,Filipino Restaurant,Food,Food & Drink Shop,Field
3,Bensonhurst,Chinese Restaurant,Sushi Restaurant,Ice Cream Shop,Italian Restaurant,Donut Shop,Liquor Store,Road,Noodle House,Cha Chaan Teng,Coffee Shop,Spa,Butcher,Grocery Store,Cosmetics Shop,Dumpling Restaurant,Pharmacy,Bakery,Smoke Shop,Shabu-Shabu Restaurant,Dessert Shop,Hotpot Restaurant,Vietnamese Restaurant,Pizza Place,Park,Bagel Shop,Bank,Field,Fish & Chips Shop,Flea Market,Filipino Restaurant
4,Bergen Beach,Harbor / Marina,Athletics & Sports,Playground,Baseball Field,Donut Shop,Hockey Field,Flea Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Women's Store,Flower Shop,Farm,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Farmers Market,Event Space,Falafel Restaurant,Factory,Frozen Yogurt Shop,Event Service,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
5,Boerum Hill,Coffee Shop,Dance Studio,Spa,Sandwich Place,Bar,Martial Arts Dojo,Kids Store,Furniture / Home Store,French Restaurant,Grocery Store,Bank,Cosmetics Shop,Bakery,Yoga Studio,Hotel,Gym / Fitness Center,Middle Eastern Restaurant,Arts & Crafts Store,Cocktail Bar,Clothing Store,Boutique,Department Store,Pharmacy,Bubble Tea Shop,Deli / Bodega,Burger Joint,Burrito Place,Seafood Restaurant,Flower Shop,Convenience Store
6,Borough Park,Bank,Pizza Place,Pharmacy,Bakery,Deli / Bodega,Eastern European Restaurant,Restaurant,Coffee Shop,Fast Food Restaurant,Food Court,Metro Station,Men's Store,Bike Shop,Toy / Game Store,Hotel,American Restaurant,Hostel,Hardware Store,Event Service,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Hockey Field,Farmers Market,Farm,Falafel Restaurant,Factory
7,Brighton Beach,Beach,Eastern European Restaurant,Restaurant,Gourmet Shop,Bank,Sushi Restaurant,Mobile Phone Shop,Russian Restaurant,Fast Food Restaurant,Bookstore,Non-Profit,Supplement Shop,Taco Place,Lounge,Mediterranean Restaurant,Korean Restaurant,Food & Drink Shop,Coffee Shop,Supermarket,Pharmacy,Grocery Store,Dessert Shop,Playgro

In [13]:
# set number of clusters
kclusters = 10

Brooklyn_grouped_clustering = Brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30]

array([6, 6, 2, 6, 9, 6, 1, 6, 0, 6, 6, 6, 5, 6, 6, 6, 6, 2, 2, 0, 6, 6,
       6, 6, 7, 7, 6, 7, 6, 7], dtype=int32)

In [14]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Brooklyn_merged = Brooklyn_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Brooklyn_merged = Brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Brooklyn_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,6,Spa,Italian Restaurant,Pizza Place,Bagel Shop,American Restaurant,Greek Restaurant,Bar,Chinese Restaurant,Sandwich Place,Middle Eastern Restaurant,Seafood Restaurant,Playground,Pharmacy,Sushi Restaurant,Ice Cream Shop,Thai Restaurant,Optical Shop,Grocery Store,Hookah Bar,Gym / Fitness Center,Snack Place,Breakfast Spot,Kids Store,Coffee Shop,Pool Hall,Clothing Store,Mobile Phone Shop,Vietnamese Restaurant,Hotel,Juice Bar
1,Brooklyn,Bensonhurst,40.611009,-73.995180,6,Chinese Restaurant,Sushi Restaurant,Ice Cream Shop,Italian Restaurant,Donut Shop,Liquor Store,Road,Noodle House,Cha Chaan Teng,Coffee Shop,Spa,Butcher,Grocery Store,Cosmetics Shop,Dumpling Restaurant,Pharmacy,Bakery,Smoke Shop,Shabu-Shabu Restaurant,Dessert Shop,Hotpot Restaurant,Vietnamese Restaurant,Pizza Place,Park,Bagel Shop,Bank,Field,Fish & Chips Shop,Flea Market,Filipino Restaurant
2,Brooklyn,Sunset Park,40.645103,-74.010316,1,Bakery,Pizza Place,Mexican Restaurant,Latin American Restaurant,Bank,Mobile Phone Shop,Pharmacy,Gym,Sandwich Place,Breakfast Spot,Stadium,Supplement Shop,Taco Place,Grocery Store,Record Shop,Women's Store,Bagel Shop,Video Game Store,Italian Restaurant,Donut Shop,Ice Cream Shop,Fish & Chips Shop,Field,Filipino Restaurant,Discount Store,Flea Market,Fish Market,Distillery,Flower Shop,Food
3,Brooklyn,Greenpoint,40.730201,-73.954241,6,Bar,Pizza Place,Coffee Shop,Cocktail Bar,Café,Bakery,Furniture / Home Store,French Restaurant,Mexican Restaurant,Record Shop,Yoga Studio,Sandwich Place,Grocery Store,Polish Restaurant,Italian Restaurant,Boutique,Sushi Restaurant,New American Restaurant,Thrift / Vintage Store,Liquor Store,Fried Chicken Joint,Burger Joint,Juice Bar,Wine Bar,Nightclub,Salon / Barbershop,Laundry Service,Restaurant,Rental Car Location,American Restaurant
4,Brooklyn,Gravesend,40.595260,-73.973471,2,Pizza Place,Lounge,Italian Restaurant,Chinese Restaurant,Bus Station,Bakery,Furniture / Home Store,Martial Arts Dojo,Baseball Field,Bar,Garden Center,Metro Station,Record Shop,Diner,Donut Shop,Pharmacy,Food & Drink Shop,Food,Flower Shop,Food Court,Flea Market,Food Stand,Fish Market,Fish & Chips Shop,Filipino Restaurant,Falafel Restaurant,Field,Electronics Store,Distillery,Dive Bar
5,Brooklyn,Brighton Beach,40.576825,-73.965094,6,Beach,Eastern European Restaurant,Restaurant,Gourmet Shop,Bank,Sushi Restaurant,Mobile Phone Shop,Russian Restaurant,Fast Food Restaurant,Bookstore,Non-Profit,Supplement Shop,Taco Place,Lounge,Mediterranean Restaurant,Korean Restaurant,Food & Drink Shop,Coffee Shop,Supermarket,Pharmacy,Grocery Store,Dessert Shop,Playground,Bakery,Convenience Store,Diner,Other Great Outdoors,Varenyky restaurant,Donut Shop,Electronics Store
6,Brooklyn,Sheepshead Bay,40.586890,-73.943186,6,Dessert Shop,Turkish Restaurant,Sandwich Place,Yoga Studio,Creperie,Miscellaneous Shop,Diner,Buffet,Boat or Ferry,Russian Restaurant,Restaurant,Karaoke Bar,Grocery Store,Café,Pizza Place,Department Store,Ice Cream Shop,American Restaurant,Hotel,Italian Restaurant,Flower Shop,Distillery,Food,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Food & Drink Shop,Fast Food Restaurant
7,Brooklyn,Manhattan Terrace,40.614433,-73.957438,1,Pizza Place,Ice Cream Shop,Coffee Sho

In [15]:
# create map
map_clusters = folium.Map(location=[40.6501038, -73.9495823], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Brooklyn_merged['Latitude'], Brooklyn_merged['Longitude'], Brooklyn_merged['Neighborhood'], Brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [24]:
#Cluster 7 is obviously the cluster with more fast food restaurants, giving us a better chance
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 7]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
10,Brooklyn,East Flatbush,40.641718,-73.936103,7,Chinese Restaurant,Park,Wine Shop,Department Store,Deli / Bodega,Moving Target,Fast Food Restaurant,Pharmacy,Caribbean Restaurant,Supermarket,Hardware Store,Bakery,Farmers Market,Field,Filipino Restaurant,Fish & Chips Shop,Women's Store,Farm,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Fish Market,Event Space,Falafel Restaurant,Factory,Food Truck,Event Service,Ethiopian Restaurant
26,Brooklyn,East New York,40.669926,-73.880699,7,Deli / Bodega,Gym,Pharmacy,Bus Station,Metro Station,Caribbean Restaurant,Fast Food Restaurant,Event Service,Convenience Store,Food Truck,Pizza Place,Donut Shop,Dry Cleaner,Food Stand,Discount Store,Food Court,Food & Drink Shop,Food,Distillery,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Dive Bar,Farmers Market,Farm,Falafel Restaurant,French Restaurant
29,Brooklyn,Flatlands,40.630446,-73.929113,7,Pharmacy,Fast Food Restaurant,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Park,Nightclub,Lounge,Bar,Electronics Store,Paper / Office Supplies Store,Sandwich Place,Video Store,Intersection,Discount Store,Farmers Market,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Filipino Restaurant,Food,Food & Drink Shop,Field,Women's Store,Event Space,Farm,Falafel Restaurant,Factory,Food Stand
47,Brooklyn,Prospect Park South,40.647009,-73.962613,7,Caribbean Restaurant,Pharmacy,Fast Food Restaurant,Grocery Store,Pizza Place,Mobile Phone Shop,Fried Chicken Joint,Latin American Restaurant,Clothing Store,Donut Shop,Chinese Restaurant,Kids Store,Electronics Store,Mexican Restaurant,Food Truck,Discount Store,Sporting Goods Shop,Supermarket,Department Store,Taco Place,Sandwich Place,Women's Store,Bar,Gym / Fitness Center,Theater,Juice Bar,Bank,Video Game Store,Falafel Restaurant,Event Space
56,Brooklyn,Rugby,40.655572,-73.926882,7,Caribbean Restaurant,Supermarket,Fried Chicken Joint,Bakery,Diner,Pharmacy,Coffee Shop,Chinese Restaurant,Sandwich Place,Fast Food Restaurant,Seafood Restaurant,Mobile Phone Shop,Gift Shop,Bank,Pizza Place,Ice Cream Shop,Wings Joint,Hardware Store,Factory,Fish Market,Fish & Chips Shop,Herbs & Spices Store,History Museum,Filipino Restaurant,Field,Hockey Field,Home Service,Farmers Market,Hookah Bar,Farm
57,Brooklyn,Remsen Village,40.652117,-73.916653,7,Caribbean Restaurant,Fast Food Restaurant,Salad Place,Donut Shop,Fish Market,Coffee Shop,Café,Supermarket,Pharmacy,Breakfast Spot,Fried Chicken Joint,Deli / Bodega,Sandwich Place,Flower Shop,Flea Market,Farm,Food,Food & Drink Shop,Fish & Chips Shop,Filipino Restaurant,Field,Farmers Market,Women's Store,Falafel Restaurant,Food Stand,Factory,Event Space,Event Service,Ethiopian Restaurant,Electronics Store
69,Brooklyn,Erasmus,40.646926,-73.948177,7,Caribbean Restaurant,Bar,Yoga Studio,Convenience Store,Mobile Phone Shop,Music Venue,Furniture / Home Store,Grocery Store,Gym / Fitness Center,Bank,Juice Bar,Food Truck,Donut Shop,Pharmacy,Playground,Pizza Place,Home Service,Dry Cleaner,Field,Food & Drink Shop,Food,Flower Shop,Distillery,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Farmers Market,Fast Food Restaurant,Dumpling Restaurant
